# Notebook for simulating and plotting spikes

This notebook is a quick introduction on how to **create**, **use**, and **analyse** neuron activity. Specifically you will learn how to simulate single neurons and plot their spikes. 

We focus on understandability, not efficiency, so note that the code could be much more efficient.

First of all we need to install some prerequisites:

In [ ]:
!pip install -q matplotlib numpy scipy tqdm

In [ ]:
!pip install -q norse

In [ ]:
import torch          # PyTorch
import numpy as np    # Numpy

# Import a simple LIF feedforward cell (no backpropagation or learning for now)
from norse.torch.module.lif import LIFCell

## Creating leaky integrate-and-fire (LIF) neurons

To start simulating we need to create a population of LIF neurons of an arbitrary size. Each neuron in the population is set to a default state. This default state tells us what the initial neuron parameters are (for now, we are just using the defaults). You can find more informations [here](https://norse.github.io/norse/auto_api/norse.torch.functional.lif.html#norse.torch.functional.lif.LIFParameters).

The neurons will receive currents as inputs, and produce spikes as outputs.

In [ ]:
# Creates a LIF population of arbitraty size
# (it will just forward any number of inputs it gets)
lif_cell = LIFCell()
print(lif_cell)

## Creating data for the neurons

To illustrate how a LIF neuron works, we wil simulate 100 timesteps. Each of the 10 inputs (indexed from 0 to 9) will sent to the same neuron at every timestep, setting the size of our previously arbitrary sized population to 10. In the setup below, the input voltages follow the formula $i_n * 0.2$, where $i_n$ is the neuron index. Meaning, neuron 0 will receive an input of 0 for each timestep, so we do not expect that to spike. Neuron 1 will receive 0.2, so that *could* be enough for it to spike. Neuron 9 will receive an input of 1.80 every timestep, so we expect that to spike quite a few times.

In [ ]:
# We are simulating 100 timesteps
timesteps = 100
# Create one current for each neuron (first neuron = 0, second neuron = 0.2, etc.)
neuron_currents = torch.tensor([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8])
# Duplicate the neuron currents 100 times (timesteps)
population_currents = neuron_currents.repeat(100, 1)
# We now have 100 timesteps of unique currents for each of the 10 neurons
population_currents.shape

## Running the simulation

To run the simulation, we prepare a list of spikes and a state-variable, where the neuron states will be updated in. During the simulation, the states will update independently, resulting in different behaviours.

In [ ]:
spikes = []  # Prepare list for spike data
state = None # Set initial state to None

# Loop over all the inputs
for data in population_currents:
    # Inject the current and the current state into the neuron
    # Returns the spikes that occurred during that timestep + the new neuron state
    out, state = lif_cell(data, state)
    # Add the spikes to our list of spike outputs
    spikes.append(out.detach().numpy())

# Convert the spike list to a Numpy array
spikes = np.array(spikes)

We now have the spike data as a list of outputs for each timestep. Let's see how it looks at timestep 100:

In [ ]:
spikes[99]

The data tells us that at timestep 100, neuron 5 and 7 emitted a spike.

## Plotting the spikes

There are likely many more spikes, but it's too much work to go through the array one timestep at the time. A spike plot is much better suited to give us an overview.

First, we import matplotlib.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Basically, we want to show the timesteps on the x axis and the neuron index on the y axis. Spikes will then be shown as a tuple of (timestep, neuron_index). If there are no spike, we don't have to show anything. 

Matplotlib's [`.eventplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.eventplot.html?highlight=eventplot#matplotlib.pyplot.eventplot) is perfect for this. We just have to condense our spikes into a list of neurons, containing the timesteps that it fires. So, for neuron 0 that list will be empty, and for neuron 9 we will have a lot of spikes. The below code arranges that data, and we can see that neuron 5 still fires at timestep 99:

In [ ]:
spike_data = [[x for x, spike in enumerate(inner_spikes) if spike > 0] for inner_spikes in spikes.T]
spike_data[5][-1] # Output the last spike from neuron number 9

We are finally ready to plot each neuron starting from 0.

In [ ]:
plt.figure(figsize=(10, 10))  # Set figure size 
plt.xlabel("Timestep")        # Label x axis
plt.ylabel("Neuron index")    # Label y axis
plt.yticks(list(range(10)))   # Force display all neuron indices
for i in range(10):           # Loop i from 0 to 9 (inclusive)
    plt.eventplot(spike_data) # Plot the timesteps where the neuron spiked

## Conclusion

And indeed, as expected neuron 0 did not fire. Nor did neuron 1. Neuron 8 and 9 however, received just the right amount of current for them to exhibit an identical firing pattern.

The power of the event-based paradigm is that this sparse behaviour can be exploited. If we look at the data over time, we can interpret this as if neuron 0 "talks less" than neuron 9. Meaning, it conveys less information.

In turn, this can be exploited in neural networks and learning (plasticity). That, however, is outside the scope of this tutorial.

## Additional resources

If you are interested in learning more, have a look [at our other notebooks](https://github.com/norse/notebooks) and [our documentation](https://norse.github.io/norse/).